In [8]:
import numpy as np
from scipy import optimize as opt

In [22]:
alpha_d = 30
alpha_r = np.deg2rad(alpha_d)
pk = 9.52
a = 0.8567 * pk
f = 0.6128 * a
L = 1.86 * pk
h1 = 0.33 * pk
h2 = 0.7 * pk
h3 = 1.115 * pk

r0 = 28
r1 = 2.6
r2 = 5.5
r3 = 1.2
r4 = 14.3
r5 = 1.8

In [96]:
# AB
def x1(t1):
    return 0.5 * a + r0 * np.cos(t1)
def x1_(t1):
    return -r0 * np.sin(t1)
def y1(t1, y0):
    return y0 + r0 * np.sin(t1)
def y1_(t1):
    return r0 * np.cos(t1)
# BC
def x2(t2, t1B, t2B):
    return x1(t1B) + r1 * np.cos(t2) - r1 * np.cos(t2B)
def x2_(t2):
    return -r1 * np.sin(t2)
def y2(t2, t1B, t2B, y0):
    return y1(t1B, y0) + r1 * np.sin(t2) - r1 * np.sin(t2B)
def y2_(t2):
    return r1 * np.cos(t2)
# CD
def x3(t3, t1B, t2B, t2C, t3C):
    return x2(t2C, t1B, t2B) + r2 * np.cos(t3) - r2 * np.cos(t3C)
def x3_(t3):
    return -r2 * np.sin(t3)
def y3(t3, t1B, t2B, t2C, t3C, y0):
    return y2(t2C, t1B, t2B, y0) + r2 * np.sin(t3) - r2 * np.sin(t3C)
def y3_(t3):
    return r2 * np.cos(t3)
# DE
def x4(t4, t3D):
    return f * np.cos(t3D) + t4 * np.sin(alpha_r)
def x4_(t3D):
    return np.sin(alpha_r)
def y4(t4, t3D):
    return f * np.sin(t3D) - t4 * np.cos(alpha_r)
def y4_(t3D):
    return - np.cos(alpha_r)
# EF
def x5(t5, t3D, t4E, t5E):
    return x4(t4E, t3D) + r3 * np.cos(t5) - r3 * np.cos(t5E)
def x5_(t5):
    return -r3 * np.sin(t5)
def y5(t5, t3D, t4E, t5E):
    return y4(t4E, t3D) + r3 * np.sin(t5) - r3 * np.sin(t5E)
def y5_(t5):
    return r3 * np.cos(t5)
# FG
def x6(t6, t3D, t4E, t5E, t5F, t6F):
    return x5(t5F, t3D, t4E, t5E) + r4 * np.cos(t6) - r4 * np.cos(t6F)
def x6_(t6):
    return -r4 * np.sin(t6)
def y6(t6, t3D, t4E, t5E, t5F, t6F):
    return y5(t5F, t3D, t4E, t5E) + r4 * np.sin(t6) - r4 * np.sin(t6F)
def y6_(t6):
    return r4 * np.cos(t6)
# GH
def x7(t7, t3D, t4E, t5E, t5F, t6F, t6G, t7G):
    return x6(t6G, t3D, t4E, t5E, t5F, t6F) + r5 * np.cos(t7) - r5 * np.cos(t7G)
def x7_(t7):
    return -r5 * np.sin(t7)
def y7(t7, t3D, t4E, t5E, t5F, t6F, t6G, t7G):
    return y6(t6G, t3D, t4E, t5E, t5F, t6F) + r5 * np.sin(t7) - r5 * np.sin(t7G)
def y7_(t7):
    return r5 * np.cos(t7)

In [122]:
def enacbe(priblizki):
    t1A, t1B, t2B, t2C, t3C, t3D, t4D, t4E, t5E, t5F, t6F, t6G, t7G, t7H, y0 = priblizki
    e1 = 0.5 * a - x1(t1A)
    e2 = h1 - y1(t1A, y0)
    e3 = x2(t2B, t1B, t2B) - x1(t1B)
    e4 = (y1_(t1B) / x1_(t1B)) - (y2_(t2B) / x2_(t2B))
    e5 = x3(t3C, t1B, t2B, t2C, t3C) - x2(t2C, t1B, t2B)
    e6 = (y2_(t2C) / x2_(t2C)) - (y3_(t3C) / x3_(t3C))
    e7 = x4(t4D, t3D) - x3(t3D, t1B, t2B, t2C, t3C)
    e8 = (y3_(t3D) / x3_(t3D)) - np.tan(alpha_r + np.pi / 2)
    e9 = x5(t5E, t3D, t4E, t5E) - x4(t4E, t3D)
    e10 = (y4_(t4E) / x4_(t4E)) - (y5_(t5E) / x5_(t5E))
    e11 = x6(t6F, t3D, t4E, t5E, t5F, t6F) - x5(t5F, t3D, t4E, t5E)
    e12 = (y5_(t5F) / x5_(t5F)) - (y6_(t6F) / x6_(t6F))
    e13 = x7(t7G, t3D, t4E, t5E, t5F, t6F, t6G, t7G) - x6(t6G, t3D, t4E, t5E, t5F, t6F)
    e14 = 0.5 * a - x7(t7H, t3D, t4E, t5E, t5F, t6F, t6G, t7G)
    # spada vmes
    e15 = 0.5 * a - 0.5 * L - x2(t2C, t1B, t2B)
    return (e1, e2, e3, e4, e5, e6, e7, e8, e9, e10, e11, e12, e13, e14, e15)

def e1_(t1):
    return - x1_(t1)
def e2_(t1):
    return - y1_(t1)
def e3_(t2):
    return x2_(t2)

In [98]:
np.deg2rad(161)

2.8099800957108707

In [136]:
zac_priblizki = [1.57, 1.81, 1.81, 3.66, 3.66, 3.9, 0, 4.5, 3.66, 5.62, 5.62, 5.93, 2.8, 1.57, -25]
#zac_priblizki = np.ones(15)
#zac_priblizki[-1] = -25
resitev = opt.fsolve(enacbe, zac_priblizki)
# resitev je oblike: (t1A, t1B, t2B, t2C, t3C, t3D, t4D, t4E, t5E, t5F, t6F, t6G, t7G, t7H, y0)

for i, element in enumerate(resitev[:-1]):
    if i == 0:
        nov_element = element - int(element / (2 * np.pi)) * 2 * np.pi
        if nov_element < 0:
            resitev[i] = nov_element + 2 * np.pi
        else:
             resitev[i] = nov_element
    elif i in (6, 7):
        pass
    else:
        nov_element = element - int(element / (2 * np.pi)) * 2 * np.pi
        if nov_element > resitev[i-1]:
            resitev[i] = nov_element
        else:
            resitev[i] = nov_element + 2 * np.pi
            
    
t1A, t1B, t2B, t2C, t3C, t3D, t4D, t4E, t5E, t5F, t6F, t6G, t7G, t7H, y0 = resitev
print(t1A, t1B, t2B, t2C, t3C, t3D, t4D, t4E, t5E, t5F, t6F, t6G, t7G, t7H, y0)
# zapis v datoteko "robne_vrednosti.txt"
with open("robne_vrednosti.txt", 'w') as robne:
    print(t1A, t1B, t2B, t2C, t3C, t3D, t4D, t4E, t5E, t5F, t6F, t6G, t7G, t7H, y0, file=robne)

1.57 1.81 8.093185307179587 9.943185307179586 9.943185307179586 10.183185307179587 0.0 4.5 9.943185307179586 11.903185307179587 11.903185307179587 12.213185307179586 9.083185307179587 7.8531853071795865 -25.0


C:\Users\Jure\AppData\Roaming\Python\Python37\site-packages\scipy\optimize\minpack.py:163: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
